In [ ]:
!pip3 install --upgrade --quiet  langchain langchain-community langchain-openai langchain-experimental neo4j wikipedia tiktoken yfiles_jupyter_graphs     

In [ ]:
!pip3 install langchain-google-genai google-generativeai

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [ ]:
from langchain_community.graphs import Neo4jGraph

In [ ]:
from dotenv import load_dotenv
load_dotenv()

import os
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

In [ ]:
NEO4J_URI="neo4j+s://2c755919.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="RGUjiPrFzwTsjFig0gc743rKroP-PqLYPc0q7fvcSEM"

In [ ]:
import os

In [ ]:
os.environ['NEO4J_URI'] = NEO4J_URI
os.environ['NEO4J_USERNAME'] = NEO4J_USERNAME
os.environ['NEO4J_PASSWORD'] = NEO4J_PASSWORD   
os.environ['GEMINI_API_KEY'] = GEMINI_API_KEY

In [ ]:
graph = Neo4jGraph()

In [ ]:
from langchain.document_loaders import WikipediaLoader
raw_documents = WikipediaLoader(query='Elizabeth I').load()

In [ ]:
len(raw_documents)

In [ ]:
from langchain.text_splitter import TokenTextSplitter
text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=24)
documents = text_splitter.split_documents(raw_documents[:3])

In [ ]:
llm = ChatGoogleGenerativeAI(api_key=GEMINI_API_KEY, model='gemini-2.0-flash-exp')

In [ ]:
!pip3 install langchain_experimental

In [ ]:
!pip3 show langchain_experimental

In [ ]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
llm_transformers = LLMGraphTransformer(llm=llm)

In [ ]:
def process_documents(documents):
    processed_docs = []
    for doc in documents:
        response = llm.invoke(f"Extract relationships and nodes from this document: {doc}")
        processed_docs.append(response)
    return processed_docs

# Call function on documents
graph_documents = process_documents(documents)
print(graph_documents)

In [ ]:
print(type(graph_documents))
print(graph_documents)

In [ ]:
graph_documents = llm.invoke(f"Convert these documents into a structured graph format: {documents}")
print(graph_documents)  # Check the response structure

In [ ]:
import json
import re

# Extract the content from AIMessage
raw_response = graph_documents.content

# Remove markdown-style code block formatting
clean_json = re.sub(r"```json\n|\n```", "", raw_response).strip()

# Try parsing the cleaned JSON
try:
    structured_data = json.loads(clean_json)
except json.JSONDecodeError as e:
    print("JSON Parsing Error:", e)
    structured_data = {"nodes": [], "relationships": []}  # Default empty structure

# Check parsed output
print(structured_data)

In [ ]:
import json

# Pretty-print to check the full extracted structure
print(json.dumps(structured_data, indent=2))

In [ ]:
graph.add_graph_documents(structured_data, baseEntityLabel=True)

In [ ]:
import json
print(json.dumps(structured_data, indent=2))

In [ ]:
for node in structured_data["nodes"]:
    if not isinstance(node.get("properties"), dict):
        node["properties"] = {}  # Ensure properties is always a dictionary
    node["properties"].setdefault("source", "Generated by Gemini")

In [ ]:
for rel in structured_data["relationships"]:
    if not isinstance(rel, dict):
        print("Error: Relationship is not a dictionary", rel)
        continue  # Skip invalid entries
    rel.setdefault("source", "Generated by Gemini")

In [ ]:
print(json.dumps(structured_data, indent=2))  # Debug the full structure

In [ ]:
structured_data.setdefault("relationships", [])  # Ensure relationships key exists

In [ ]:
# Validate nodes
for node in structured_data["nodes"]:
    if not isinstance(node.get("properties"), dict):
        node["properties"] = {}  # Ensure properties is always a dictionary
    node["properties"].setdefault("source", "Generated by Gemini")  # Default source

# Validate relationships
if "relationships" in structured_data:  # Check if key exists
    for rel in structured_data["relationships"]:
        if not isinstance(rel, dict):
            print("Error: Relationship is not a dictionary", rel)
            continue  # Skip invalid entries
        rel.setdefault("source", "Generated by Gemini")

In [ ]:
graph.add_graph_documents(structured_data, baseEntityLabel=True)

In [ ]:
from langchain_community.graphs.graph_document import GraphDocument

In [ ]:
from langchain_community.graphs.graph_document import GraphDocument

# Ensure structured_data has the required keys
structured_data.setdefault("nodes", [])
structured_data.setdefault("relationships", [])

graph_documents = []

# Convert nodes to GraphDocument objects
for node in structured_data["nodes"]:
    graph_documents.append(
        GraphDocument(
            page_content=str(node.get("id", "Unknown Entity")),  # Ensure page_content is a string
            metadata={"label": node.get("label", "Unknown"), "properties": node.get("properties", {})},
            source={"source": node.get("properties", {}).get("source", "Generated by Gemini")}  # Ensure source is a valid dictionary
        )
    )

# Convert relationships to GraphDocument objects
for rel in structured_data["relationships"]:
    graph_documents.append(
        GraphDocument(
            page_content=f"{rel.get('source', 'Unknown')} -[{rel.get('relation', 'Unknown')}]→ {rel.get('target', 'Unknown')}",
            metadata={"relation": rel.get("relation", "Unknown"), "source": rel.get("source", "Unknown"), "target": rel.get("target", "Unknown")},
            source={"source": "Generated by Gemini"}  # Ensure source is a valid dictionary
        )
    )

# Add to Neo4j graph
graph.add_graph_documents(graph_documents, baseEntityLabel=True)